In [1]:
import numpy as np
import cv2 

In [2]:


def kuwahara_filter(image, kernel_size=5):

    height, width, channel = image.shape[0], image.shape[1], image.shape[2]
    r = int((kernel_size - 1) / 2)
    if r>= 2:
        r = r
    else:
        r = 2
    image = np.pad(image, ((r, r), (r, r), (0, 0)), "edge")
    
    average, variance = cv2.integral2(image)
    average = (average[:-r - 1, :-r - 1] + average[r + 1:, r + 1:] -
               average[r + 1:, :-r - 1] - average[:-r - 1, r + 1:]) / (r +
                                                                       1)**2
    variance = ((variance[:-r - 1, :-r - 1] + variance[r + 1:, r + 1:] -
                 variance[r + 1:, :-r - 1] - variance[:-r - 1, r + 1:]) /
                (r + 1)**2 - average**2).sum(axis=2)

    def filter(i, j):
        return np.array([
            average[i, j], average[i + r, j], average[i, j + r], average[i + r,
                                                                         j + r]
        ])[(np.array([
            variance[i, j], variance[i + r, j], variance[i, j + r],
            variance[i + r, j + r]
        ]).argmin(axis=0).flatten(), j.flatten(),
            i.flatten())].reshape(width, height, channel).transpose(1, 0, 2)

    filtered_image = filter(*np.meshgrid(np.arange(height), np.arange(width)))

    filtered_image = filtered_image.astype(image.dtype)
    filtered_image = filtered_image.copy()

    return filtered_image


In [3]:
#read image
img = cv2.imread('./img.jpg')
#filter image
filtered_img = kuwahara_filter(img)
#create new image
cv2.imshow('img',filtered_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('new_img.jpg', filtered_img)
imgGray = cv2.cvtColor(filtered_img, cv2.COLOR_BGR2GRAY)
cv2.imshow('imgGray',imgGray)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('new_imgGray.jpg', imgGray)



True